# <font color='#30a54f'>**mantis**</font>  <font color='black'>**geophysics**</font>
An Edinburgh University spinout - contact: gpap@mantis-geophysics.com
### Author: G. Papageorgiou
Utilities - In this notebook we showcase the palette of utilities for various purposes in the mantis codebase.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import mantis_core.utilities as manUT
import mantis_core._literature as manLit


First we generate cijs with different symmetries. There is no rock physics modelling taking place here, we just read values for elastic tensors from the literature

In [2]:
background = {"Vp":3.4, "Vs":1.8, "Rho":2.6}
isocij = manUT.VtoCij(**background) # a simple isotropic Cij with elements calculated from Vp, Vs, Rho
ticij = manUT.thomsen_vti(**background, epsilon = 0.1, delta = 0.1, gamma = 0.1) # as above, but with Thomsen VTI anisotropy
tiltedTIcij = manUT.tilt_rotation(ticij, theta = np.pi/8.) # as above, but rotated by pi/8 along the y-axis
HTIcij = manUT.tilt_rotation(
    ticij, theta=np.pi / 2
)  # HTI
ortcij = manLit.Examples.presets["Jin & Stovas, gji-2021"]["medium-1"]["Cij"] # a simple orthorhombic Cij from Jin & Stovas, gji-2021
tiltcij = manLit.Examples.presets["Generic testing"]["medium-1"]["Cij"] # a simple tilted orthorhombic Cij from Jin & Stovas, gji-2021
monTilted = ortcij + tiltedTIcij # adding the tilted ti to the orthorhombic we get a tilted orthorhombic
mono = ortcij + manUT.interchange_y_and_z(
    tiltedTIcij
)  # changing the tilt to be in the x-z plane instead of the x-y plane so that we get an U/D symmetry

A convenient utility checks the symmetry of the elastic tensors:

In [3]:
Cij = np.array(
    [
        [36.07, 15.85, 16.03, 0.0, 0.0, 0.0],
        [15.85, 36.07, 16.03, 0.0, 0.0, 0.0],
        [16.03, 16.03, 30.06, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 8.42, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 8.42, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 10.11],
    ]
)
manUT.tilt_rotation(Cij, theta=np.pi / 8.0).round(2)

array([[35.14, 15.88, 16.08,  0.  ,  1.11,  0.  ],
       [15.88, 36.07, 16.  ,  0.  , -0.06,  0.  ],
       [16.08, 16.  , 30.89,  0.  ,  1.01,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  8.67,  0.  ,  0.6 ],
       [ 1.11, -0.06,  1.01,  0.  ,  8.47,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.6 ,  0.  ,  9.86]])

In [4]:
manUT.check_symmetry(Cij, 5)

('U/D-symmetric', 'Transversely Isotropic')

In [5]:
for cij in [isocij, ticij, tiltedTIcij, HTIcij, ortcij, tiltcij, mono, monTilted]:
    print(manUT.check_symmetry(cij, 5))

('U/D-symmetric', 'Isotropic')
('U/D-symmetric', 'Transversely Isotropic')
('Tilted', 'Transversely Isotropic')
('U/D-symmetric', 'Transversely Isotropic')
('U/D-symmetric', 'Orthorhombic')
('Tilted', 'Orthorhombic')
('U/D-symmetric', 'Monoclinic')
('Tilted', 'Monoclinic')


This works with any elastic tensor in the form of a 6x6 matrix.

In [6]:
import numpy as np
random_matrix = 50*np.random.random(size=(6,6))
random_cij = (random_matrix + random_matrix.T) / 2
manUT.check_symmetry(random_cij) # would be triclinic and tilted (whatever that means!)

('Tilted', 'Triclinic')

Finally, we have isotropic and anisotropic conversions of incidence angles to slowness and back

In [7]:
?manUT.incidence_angle_to_slowness

Signature: manUT.incidence_angle_to_slowness(incidence_angle=0.0, Vp: 'float' = None)
Docstring:
Convert incidence angle to slowness.

Parameters
----------
incidence_angle : float, optional
    The incidence angle in radians. Default is 0.0.
Vp : float, optional
    The P-wave velocity. Default is None.

Returns
-------
slowness : float
    The slowness corresponding to the given incidence angle in an isotropic medium with P-wave velocity `Vp`.

Notes
-----
The slowness is calculated by the formula:

$$
s = \frac{\sin(\theta)}{V_p}
$$
File:      ~/Documents/projects/mantis-core/src/mantis_core/utilities.py
Type:      function

In [8]:
?manUT.slowness_to_incidence_angle

Signature: manUT.slowness_to_incidence_angle(slowness=0.0, Vp=1.0) -> 'float'
Docstring:
 Convert slowness to incidence angle.

 Parameters
 ----------
 slowness : float, optional
     The slowness value. Defaults to 0.0.
 Vp : float, optional
     The P-wave velocity. Defaults to 1.0.

 Returns
 -------
 float
     The incidence angle in radians.

 Notes
 -----
 The incidence angle is calculated by the formula:

$$
\theta = \arcsin(s V_p)
$$
File:      ~/Documents/projects/mantis-core/src/mantis_core/utilities.py
Type:      function